<a href="https://colab.research.google.com/github/vishakhun/Data_Analysis/blob/main/ESG_Risk_Dashboard_Development_Using_Open_Source_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ESG Risk Dashboard Development Using Open-Source Data**
##**Project Overview**
This project aims to develop an interactive ESG (Environmental, Social, and Governance) Risk Dashboard using open-source data from the World Bank's World Development Indicators. The dashboard visualizes key ESG indicators, providing insights into CO2 emissions, electricity access, and unemployment rates across different countries.

The project is structured as follows:

* **Data Acquisition**: Fetching ESG indicators from the World Bank's API.
* **Data Processing**: Cleaning and preparing the data for visualization.
* **Dashboard Development**: Creating an interactive dashboard using Dash and Plotly.
* **Additional Visualizations**: Generating supplementary visualizations for deeper insights.

#**1. Data Acquisition and Processing**


In [ ]:
%%capture
#install necessary libraries
!pip install wbdata
!pip install dash

In [ ]:
#import necessary libraries
import wbdata
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In this section, we're defining the specific ESG (Environmental, Social, and Governance) indicators that we wish to analyze and visualize. We've selected three key indicators that provide insights into the environmental and social aspects of different countries:

* **1.CO2 emissions (EN.ATM.CO2E.PC)**: This indicator represents the carbon dioxide emissions in metric tons per capita. It's a crucial metric for understanding a country's environmental impact and its contribution to climate change.
* **2.Electricity access (EG.ELC.ACCS.ZS)**: This metric indicates the percentage of the population with access to electricity. It's an essential indicator of social development and economic progress.
* **3.Unemployment (SL.UEM.TOTL.ZS)**: This indicator shows the percentage of the total labor force that is unemployed but seeking employment. It provides insight into the economic conditions and labor market of a country.

After defining these indicators, we fetch the data from the World Bank's open-source database using the wbdata library. The data is then cleaned and processed to remove any missing values. We also filter the dataset to include data points within the time frame from January 1, 2010, to January 1, 2020, to focus our analysis on the most recent and relevant data.



In [ ]:
# Indicators to fetch
indicators = {
    "EN.ATM.CO2E.PC": "CO2 emissions",
    "EG.ELC.ACCS.ZS": "Electricity access",
    "SL.UEM.TOTL.ZS": "Unemployment"
}

# Fetching and processing the data
df = wbdata.get_dataframe(indicators, country='all')
df = df.dropna().reset_index()

# Filtering data
df['date'] = pd.to_datetime(df['date'])
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2020-01-01')
df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

# Show a sample of the processed data
df

,country,date,CO2 emissions,Electricity access,Unemployment
0,Africa Eastern and Southern,2020-01-01,0.795420,46.268621,7.910291
1,Africa Eastern and Southern,2019-01-01,0.915294,44.389773,7.426777
2,Africa Eastern and Southern,2018-01-01,0.921453,43.028332,7.237563
3,Africa Eastern and Southern,2017-01-01,0.933874,40.197332,7.274298
4,Africa Eastern and Southern,2016-01-01,0.941337,38.851444,7.152157
...,...,...,...,...,...
5893,Zimbabwe,2014-01-01,0.866838,32.300000,4.770000
5894,Zimbabwe,2013-01-01,0.901248,40.644161,4.932000
5895,Zimbabwe,2012-01-01,0.901214,44.000000,5.020000
5896,Zimbabwe,2011-01-01,0.871932,36.900000,5.370000


#**2. Dashboard Development**
##Dashboard Interaction
**Country Selection Dropdown**: At the top/bottom of the dashboard, there's a dropdown menu labeled 'Select Country.' Click on the dropdown to see a list of available countries. Select any country to update the graphs with data specific to that country.

**Time-Series Graph**: Below/above the dropdown, there's a time-series graph displaying the selected ESG indicators (CO2 emissions, Electricity access, and Unemployment) over time. The x-axis represents time (years), and the y-axis represents the values of the indicators. Hover over the graph to see the exact values at different points in time.

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    dcc.Graph(id='esg-graph'),
    html.Label('Select Country:'),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df['country'].unique()],
        value='United States'
    )
])

# Callback to update the graph based on the dropdown selection
@app.callback(
    Output('esg-graph', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_figure(selected_country):
    filtered_df = df[df['country'] == selected_country].dropna()

    # Creating the figure using Plotly Express
    import plotly.express as px
    fig = px.line(filtered_df, x='date', y=['CO2 emissions', 'Electricity access', 'Unemployment'],
                  labels={'value': 'Indicator Value', 'variable': 'Indicators'})

    fig.update_layout(title=f'ESG Indicators for {selected_country}')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

#**3. Additional Visualizations**

##**CO2 Emissions Comparison - 2020**

**Bar Chart Interaction**: This chart shows the comparison of CO2 emissions across different countries in 2020. Hover over each bar to see the specific CO2 emissions value for that country. The x-axis lists the countries, and the y-axis shows the CO2 emissions values.You can an also select an area to zoom in.

In [ ]:
# Generate a sample comparison for a specific year 2020
comparison_df = df[df['date'] == '2020'].groupby('country').mean().reset_index()

fig = px.bar(comparison_df, x='country', y='CO2 emissions', title='CO2 Emissions Comparison - 2020')
fig.show()

<ipython-input-5-5f8bdcdf56e8>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



##**ESG Indicator Correlation Heatmap**

**Heatmap Interaction**: The heatmap displays the correlation between different ESG indicators. A value close to 1 indicates a strong positive correlation, while a value close to -1 indicates a strong negative correlation. Hover over each cell to see the exact correlation value between the indicators.

In [ ]:
# Assuming df is your cleaned and processed DataFrame
corr_matrix = df[['CO2 emissions', 'Electricity access', 'Unemployment']].corr()

fig = px.imshow(corr_matrix,
                labels=dict(color="Correlation"),
                x=['CO2 emissions', 'Electricity access', 'Unemployment'],
                y=['CO2 emissions', 'Electricity access', 'Unemployment'],
                text_auto=True)

fig.update_layout(title='ESG Indicator Correlation Heatmap', xaxis_nticks=36)
fig.show()

##**Global ESG Indicators in 2020**

**Tips for Interacting with the Dashboard**

**Zoom and Pan**: For the maps, you can zoom in and out to focus on specific regions. You can also click and drag to pan across the map.

**Refreshing Data**: If you change the country selection in the dashboard, give it a moment to refresh the data and update the graph accordingly.

**Detailed View**: For a more detailed view, you can expand the visualization to full screen by clicking on the expand button in the top right corner of each graph.


##CO2 Emissions

**Map Interaction**: This map visualizes global CO2 emissions. Countries are colored based on their CO2 emissions levels, with the color scale provided for reference. Hover over a country to see its name and the specific CO2 emissions value.

In [ ]:
# Filter the dataframe for the year 2020
map_df = df[df['date'].dt.year == 2020]

fig = px.choropleth(
    map_df,
    locations="country",
    locationmode='country names',
    color="CO2 emissions",
    hover_name="country",
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Global CO2 Emissions in 2023'
)
fig.show()

##Electricity Access

**Map Interaction**: Similar to the CO2 emissions map, this visualization shows the percentage of electricity access across different countries. Hover over a country to view its name and the electricity access percentage.

In [ ]:
# Filter the dataframe for the year 2020
map_df = df[df['date'].dt.year == 2020]

fig = px.choropleth(
    map_df,
    locations="country",
    locationmode='country names',
    color="Electricity access",
    hover_name="country",
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Global Electricity Access in 2020'
)
fig.show()

##Unemployment
**Map Interaction**: This map displays the unemployment rates across various countries. Hover over any country to see its name and the unemployment rate.

In [ ]:
# Filter the dataframe for the year 2020
map_df = df[df['date'].dt.year == 2020]

fig = px.choropleth(
    map_df,
    locations="country",
    locationmode='country names',
    color="Unemployment",
    hover_name="country",
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Global Unemployment in 2020'
)
fig.show()

## Conclusion

In this project, we explored Environmental, Social, and Governance (ESG) indicators across various countries using open-source data from the World Bank. Our analysis focused on three key indicators: **CO2 emissions**, **electricity access**, and **unemployment rates**.

- **CO2 Emissions**: The visualizations provided insights into the global distribution of CO2 emissions, highlighting the countries with the highest and lowest emissions per capita. Understanding these patterns is crucial for addressing global climate change challenges.

- **Electricity Access**: By examining the electricity access rates, we identified regions where significant progress has been made and areas that still face challenges in providing universal access to electricity. This indicator is vital for assessing a country's development and quality of life.

- **Unemployment Rates**: The analysis of unemployment rates shed light on the economic conditions and labor markets of different countries. Identifying trends and outliers can help policymakers and stakeholders formulate strategies to enhance employment opportunities.

The interactive dashboard allowed us to delve deeper into the data, enabling a dynamic exploration of how these indicators vary across countries and over time. By interacting with the dashboard, users can gain a more nuanced understanding of the ESG landscape.

### Insights and Implications

- The disparities in CO2 emissions underscore the need for targeted environmental policies, especially in high-emission countries.
- The electricity access data highlights the progress in infrastructure development, yet it also emphasizes the ongoing need to address energy poverty.
- Analyzing unemployment trends is crucial for understanding economic stability and the effectiveness of labor market policies.

### Future Directions

Moving forward, this project can be expanded by incorporating additional ESG indicators, applying predictive analytics for forecasting trends, and enhancing the dashboard with more interactive features. By continuing to analyze and visualize these critical metrics, we can contribute to informed decision-making and sustainable development efforts globally.
